In [38]:
import cv2
import numpy as np
import random

# Load the ground truth image (replace 'ground_truth.png' with your image file)
ground_truth = cv2.imread('/home/somayeh/PycharmProjects/superpixel_segmentation/archive (3)/data/Lung Segmentation/masks/CHNCXR_0001_0.png', 0)
ground_truth = cv2.resize(ground_truth, (352, 352))
cv2.imshow("gt", ground_truth)
cv2.waitKey(0)

# Define the foreground (white pixels) and background (black pixels)
foreground_value = 255  # White pixels
background_value = 0  # Black pixels

# Threshold to obtain the foreground mask
_, foreground_mask = cv2.threshold(ground_truth, background_value + 1, 255, cv2.THRESH_BINARY)

# Find contours in the foreground mask
contours, _ = cv2.findContours(foreground_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a blank mask to draw contours
contour_mask = np.zeros_like(foreground_mask)

# Draw contours on the mask
cv2.drawContours(contour_mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

# Define the structuring element for dilation
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))  # Example kernel (adjust size as needed)

# Dilate the contour mask multiple times with randomly chosen kernels
num_dilation_iterations = 5
for i in range(num_dilation_iterations):
    
    dilated_image = cv2.dilate(contour_mask, kernel)
    
cv2.imshow("Dilation image", dilated_image)
cv2.waitKey(0)

# Initialize counters for boundaries kept and eroded
boundaries_kept = 0
boundaries_eroded = 0

# Define the number of iterations for dilation and erosion
num_erosion_iterations = 2

# Threshold to obtain the foreground mask
_, foreground_mask_1 = cv2.threshold(dilated_image, background_value + 1, 255, cv2.THRESH_BINARY)

# Find contours in the foreground mask
contours_1, _ = cv2.findContours(foreground_mask_1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a blank mask to draw contours
contour_mask_1 = np.zeros_like(foreground_mask_1)

# Draw contours on the mask
cv2.drawContours(contour_mask_1, contours_1, -1, (255, 255, 255), thickness=cv2.FILLED)

# Loop through erosion iterations
for i in range(num_erosion_iterations):
    random_prob = random.uniform(0, 1)
    
    if 0 <= random_prob < 0.5:  # Keep the boundary
        boundaries_kept += np.sum(contour_mask_1 == 255)
    else:  # Perform erosion with a randomly selected kernel size
        erosion_kernel_size = random.randint(1, 5)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (erosion_kernel_size, erosion_kernel_size))
        contour_mask_1 = cv2.erode(contour_mask_1, kernel)
        boundaries_eroded += np.sum(contour_mask_1 == 255)

cv2.imshow("erode image", contour_mask_1)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Display the counts of boundaries kept and eroded
print(f"Boundaries Kept: {boundaries_kept}")
print(f"Boundaries Eroded: {boundaries_eroded}")


Boundaries Kept: 31635
Boundaries Eroded: 30519
